In [9]:
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
import pyproj

from shapely.geometry import mapping, shape, Point, Polygon, LineString

In [2]:
def get_access_token(consumer_key, consumer_secret) -> None :
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]

    return access_token


consumer_key = "0557bd5252b34142911d"
consumer_secret = "d90d86911924440786d4"

print('access token: ', get_access_token(consumer_key, consumer_secret))

access token:  991aca35-a93a-4d88-98a9-6f24ad0df500


In [5]:
## SGIS access_token 가져오기
access_token = get_access_token(consumer_key, consumer_secret)

## 주소를 입력해 주소의 데이터를 DataFrame으로 가지고오는 함수
def get_adress_info(access_token, address) -> None:
    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    temp_df = pd.DataFrame(temp_res)

    return temp_df

# GeoPandas 를 활용한 좌표계 변환 방법

In [7]:
# 인천공항 주소로 데이터 가져오기
address_df = get_adress_info(access_token, "인천 중구 공항로 271")
address_df

,ri_nm,road_nm_sub_no,sgg_cd,adm_cd,road_nm_main_no,leg_cd,road_nm,bd_matches,bd_sub_nm,addr_type,...,road_cd,bd_main_nm,adm_nm,jibun_main_no,origin_xy,jibun_sub_no,ri_cd,y,leg_nm,x
0,null,0,23010,23010640,271,2811014700,공항로,4,null,6,...,3149003,인천국제공항역,운서동,null,BUILD,null,2811014700,1939236.69257386,운서동,907316.512763601


In [8]:
address_df['x'] = address_df['x'].astype(float)
address_df['y'] = address_df['y'].astype(float)
address_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ri_nm            1 non-null      object 
 1   road_nm_sub_no   1 non-null      object 
 2   sgg_cd           1 non-null      object 
 3   adm_cd           1 non-null      object 
 4   road_nm_main_no  1 non-null      object 
 5   leg_cd           1 non-null      object 
 6   road_nm          1 non-null      object 
 7   bd_matches       1 non-null      object 
 8   bd_sub_nm        1 non-null      object 
 9   addr_type        1 non-null      object 
 10  sido_nm          1 non-null      object 
 11  sgg_nm           1 non-null      object 
 12  sido_cd          1 non-null      object 
 13  road_cd          1 non-null      object 
 14  bd_main_nm       1 non-null      object 
 15  adm_nm           1 non-null      object 
 16  jibun_main_no    1 non-null      object 
 17  origin_xy        1 n

In [10]:
address_df['geometry'] = address_df.apply(lambda row: Point([row['x'], row['y']]), axis=1)
address_geo_df = gpd.GeoDataFrame(address_df, geometry="geometry")
address_geo_df[['bd_main_nm', 'x', 'y', 'geometry']]

,bd_main_nm,x,y,geometry
0,인천국제공항역,907316.512764,1.939237e+06,POINT (907316.513 1939236.693)


In [11]:
# 좌표계를 확인했을때 None면 정의 해줘야함
print(address_geo_df.crs)

None


In [12]:
address_geo_df.crs = {"init":"epsg:5179"} # epsg:5179 는 UTM-K 좌표계
address_geo_df.crs

C:\Users\김영준\PycharmProjects\aiffel_study\venv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


<Projected CRS: +init=epsg:5179 +type=crs>
Name: Korea 2000 / Unified CS
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Republic of Korea (South Korea) - onshore and offshore.
- bounds: (122.71, 28.6, 134.28, 40.27)
Coordinate Operation:
- name: Korea Unified Belt
- method: Transverse Mercator
Datum: Geocentric datum of Korea
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [13]:
# UTM-K 좌표계를 WGS84(위도,경도)좌표계로 전환
address_geo_df = address_geo_df.to_crs({"init":"epsg:4326"})
address_geo_df[['bd_main_nm', 'x', 'y', 'geometry']]

C:\Users\김영준\PycharmProjects\aiffel_study\venv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,bd_main_nm,x,y,geometry
0,인천국제공항역,907316.512764,1.939237e+06,POINT (126.45217 37.44768)


# pyproj를 활용한 좌표계 변환

In [14]:
# 국립 현충원 데이터 가져오기
address_df2 = get_adress_info(access_token, '서울특별시 동작구 동작동 271-18')
address_df2[['sido_nm', 'sgg_nm', 'adm_nm', 'jibun_main_no', 'jibun_sub_no', 'x', 'y']]

,sido_nm,sgg_nm,adm_nm,jibun_main_no,jibun_sub_no,x,y
0,서울특별시,동작구,사당2동,271,18,953475.620000162,1944631.15909765


In [15]:
# pyproj의 Proj, transform 메소드를 이용
def utmk_to_wgs84(utmk_x, utmk_y) -> None:
    utmk_proj = pyproj.Proj(init="EPSG:5179") # UTM-K 좌표계 정의
    wgs84_proj = pyproj.Proj(init="EPSG:4326") # WGS84 좌표계 정의

    wgs84_x, wgs84_y = pyproj.transform(utmk_proj, wgs84_proj, utmk_x, utmk_y)

    return wgs84_x, wgs84_y

In [17]:
# 변환된 좌푯값을 DataFrame 컬럼에 추가
address_df2[['wgs84_x', 'wgs84_y']] = utmk_to_wgs84(address_df2.loc[0,'x'], address_df2.loc[0,'y'])
address_df2[["sido_nm", "sgg_nm", "adm_nm", "jibun_main_no", "jibun_sub_no", "x", "y", "wgs84_x", "wgs84_y"]]

C:\Users\김영준\PycharmProjects\aiffel_study\venv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\김영준\PycharmProjects\aiffel_study\venv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\김영준\AppData\Local\Temp\ipykernel_15144\2173025951.py:6: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading

,sido_nm,sgg_nm,adm_nm,jibun_main_no,jibun_sub_no,x,y,wgs84_x,wgs84_y
0,서울특별시,동작구,사당2동,271,18,953475.620000162,1944631.15909765,126.973651,37.499772


# SGIS API를 활용한 좌표계 변환

In [18]:
# 서울 시청 데이터
address_df3 = get_adress_info(access_token, "서울 중구 세종대로 110 서울특별시청")
address_df3[["sido_nm", "sgg_nm", "adm_nm", "jibun_main_no", "jibun_sub_no", "x", "y"]]

,sido_nm,sgg_nm,adm_nm,jibun_main_no,jibun_sub_no,x,y
0,서울특별시,중구,명동,null,null,953893.87330747,1952017.73064447


In [19]:
# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(access_token, src, dst, posX, posY) -> None:
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"]

In [20]:
# 변환 후 변환된 데이터 칼럼 추가
address_df[["transcoord_x", "transcoord_y"]] = get_transcoord(access_token, "EPSG:5179", "EPSG:4326", address_df.loc[0, "x"], address_df.loc[0, "y"])
address_df[["sido_nm", "sgg_nm", "adm_nm", "jibun_main_no", "jibun_sub_no", "x", "y", "transcoord_x", "transcoord_y"]]

,sido_nm,sgg_nm,adm_nm,jibun_main_no,jibun_sub_no,x,y,transcoord_x,transcoord_y
0,인천광역시,중구,운서동,null,null,907316.512764,1.939237e+06,126.452173,37.447676
